This is a simple Julia 0.6 notebook. Below we'll show off the Julia `PyPlot` module.

In [62]:
d=4;
S=5;
T=6;
R=7;

In [63]:
Kx=rand(d,R);
Ky=rand(d,R);
Ks=rand(S,R);
Kt=rand(T,R);
U=rand(d,d,S);

K=zeros(d,d,S,T);
V=zeros(T);

In [64]:
# build up K
for i=1:d
    for j=1:d
        for s=1:S
            for t=1:T
                for r=1:R
                    K[i,j,s,t] += Kx[i,r]*Ky[j,r]*Ks[s,r]*Kt[t,r];
                end
            end
        end
    end
end


In [65]:
# equation 3 slow way
tic()
for t=1:T
    for i=1:d
        for j=1:d
            for s=1:S
                V[t] += K[i,j,s,t]*U[i,j,s];
            end
        end
    end
end
toc()


0.079794161

elapsed time: 0.079794161 seconds


In [71]:
Us=zeros(d,d,R);
Usy=zeros(d,R);
Usyx=zeros(R);
Vfast=zeros(T);

tic()

# this can be a matrix multiply after some reshapes
# Ur = reshape(U,(d*d,S));
# UrKs = Ur*Ks;
# Us = reshape(UrKs,(d,d,R));
# or all at once if it saves some allocations
Us=reshape(reshape(U,(d*d,S))*Ks,(d,d,R));

# this one is trickier, just make sure loop is cache efficient I guess
for i=1:d
    for j=1:d
        for r=1:R
            Usy[i,r] += Ky[j,r]*Us[i,j,r];
        end
    end
end

# these are just a bunch of inner products
for r=1:R
    Usyx[r] = Kx[:,r]'*Usy[:,r]
end

# matrix vector product
Vfast=Kt*Usyx

toc()

norm(V-Vfast)

8.881784197001252e-15

elapsed time: 0.061927717 seconds
